In [1]:
# === 1. Importar librerías necesarias ===
import warnings
warnings.filterwarnings("ignore")
import os
import json
import random  # Random for generating random numbers
import torch   # PyTorch for building and training the neural network
import numpy as np  # Numpy for data manipulation and generation of the sine wave
import pandas as pd  # Pandas for easy data manipulation and analysis
import seaborn as sns
import torch.nn as nn  # nn contains the modules for neural network layers and operations
from tqdm import tqdm
import statsmodels.api as sm
from scipy.stats import shapiro
import matplotlib.pyplot as plt  # Matplotlib for plotting the results (optional)
from sklearn.impute import SimpleImputer
from statsmodels.tsa.stattools import adfuller
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from torch.utils.data import TensorDataset, DataLoader
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.stats.diagnostic import het_breuschpagan
from sklearn.feature_selection import mutual_info_regression
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [2]:
import sys
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Lectura de los datos originales

In [3]:
# === 2. Cargar tu archivo completo ===
# Establecer ruta del archivo
ruta = '/content/drive/MyDrive/Colab Notebooks/Energy thesis/Pytorch/Codigos finales tesis/Bases de datos originales/'
archivo = 'Matriz_energetica_economica.xlsx'

In [5]:
# Lectura y conversión a un panda data frame
X = pd.read_excel(ruta+archivo)

In [6]:
X.head()

,t,PIB a precios corrientes (Millones de pesos),"Producción bruta del sector energetico, (Millones de pesos a precios corrientes).",Producción de petróleo crudo (Miles de barriles por día),Producción de gas natural (Millones de pies cúbicos diarios),"Producción de hidrocarburos liquidos, (Miles de barriles diarios)",Exportaciones de petróleo crudo (Miles de barriles diarios),"Exportacion de petroleo crudo, (Miles de dolares )","Importación de mercancias, Petroleras, (Millones de dólares)",Balanza comercial petrolera (Millones de dolares),...,"Generación bruta de energia eléctrica, Renovable (Gigawatts/Hora)",Tasa de crecimento de sector energetico,"Tasa de crecimento de sector energetico , energias renovables","Tasa de crecimento de sector energetico, Eolica","Tasa de crecimento de sector energetico, Geotermica",Tasa de Desempleo/%,Gasto Publico,Esperanza de Vida,Salario Minimo/ $,Consumo Privado
0,1996,14499128.96,47497.6,2861,4194.9,2858.31,1552.0,984742.58,256019.08,728723.50,...,38580,4084129498,7970446658,-1.666667e+09,1.058388e+09,5.5,581532.9,72.0,20.15,8916935.26
1,1997,15542903.30,59150.5,3025,4467.1,3021.93,1731.0,954619.67,367344.75,587274.92,...,33163,2453366065,-1404095386,-2.000000e+01,-4.590679e+09,3.7,747297.7,72.5,26.45,9612374.40
2,1998,16504244.13,70075.7,3073,4790.7,2505.21,1738.0,607980.42,353813.33,254167.08,...,31815,1847017354,-406477098,5.000000e+01,3.494329e+09,3.2,823503.4,72.6,30.20,10274171.04
3,1999,16958941.08,94046.4,2909,4790.5,2906.36,1551.0,829956.92,402871.00,427085.92,...,39712,3420686486,2482162502,1.666667e+09,-6.010253e-01,2.5,1013904.9,73.2,34.45,10710868.68
4,2000,17811854.38,103213.7,3015,4679.0,3011.85,1620.0,1343692.67,664461.50,679231.17,...,40401,974763521,1734991942,4.285714e+09,4.748355e+09,2.2,1238144.2,73.6,37.90,11628028.20


In [7]:
X.shape

(28, 68)

In [ ]:
# Establecer como índice temporal la columna 'Fecha'

In [8]:
dia = 31
mes = 12

X['fecha'] = pd.to_datetime({
    'year': X['t'],
    'month': mes,
    'day': dia,
})

X.set_index('fecha', inplace=True)

In [9]:
# Con el índice temporal establecido ya podemos eliminar la columna 'Fecha'
X.drop(columns=['t'], inplace=True)

In [10]:
# Convertir todos los valores de las series a númericas
X = X.apply(pd.to_numeric, errors='coerce')

In [11]:
print("¿Todo es numérico?", X.dtypes.apply(lambda x: np.issubdtype(x, np.number)).all())


¿Todo es numérico? True


In [12]:
# Limpiar nombres de columnas
X.columns = X.columns.str.strip()
# Guardar los nombres de las columnas en una lista
columnas_full = X.columns.tolist()

In [13]:
X.shape

(28, 67)

In [ ]:
X.head()

,PIB a precios corrientes (Millones de pesos),"Producción bruta del sector energetico, (Millones de pesos a precios corrientes).",Producción de petróleo crudo (Miles de barriles por día),Producción de gas natural (Millones de pies cúbicos diarios),"Producción de hidrocarburos liquidos, (Miles de barriles diarios)",Exportaciones de petróleo crudo (Miles de barriles diarios),"Exportacion de petroleo crudo, (Miles de dolares )","Importación de mercancias, Petroleras, (Millones de dólares)",Balanza comercial petrolera (Millones de dolares),"índice de volumne fisico, electricidad, gas y agua (indices)",...,"Generación bruta de energia eléctrica, Renovable (Gigawatts/Hora)",Tasa de crecimento de sector energetico,"Tasa de crecimento de sector energetico , energias renovables","Tasa de crecimento de sector energetico, Eolica","Tasa de crecimento de sector energetico, Geotermica",Tasa de Desempleo/%,Gasto Publico,Esperanza de Vida,Salario Minimo/ $,Consumo Privado
fecha,,,,,,,,,,,,,,,,,,,,,
1996-12-31,14499128.96,47497.6,2861,4194.9,2858.31,1552.0,984742.58,256019.08,728723.50,37.2,...,38580,4084129498,7970446658,-1.666667e+09,1.058388e+09,5.5,581532.9,72.0,20.15,8916935.26
1997-12-31,15542903.30,59150.5,3025,4467.1,3021.93,1731.0,954619.67,367344.75,587274.92,37.1,...,33163,2453366065,-1404095386,-2.000000e+01,-4.590679e+09,3.7,747297.7,72.5,26.45,9612374.40
1998-12-31,16504244.13,70075.7,3073,4790.7,2505.21,1738.0,607980.42,353813.33,254167.08,34.8,...,31815,1847017354,-406477098,5.000000e+01,3.494329e+09,3.2,823503.4,72.6,30.20,10274171.04
1999-12-31,16958941.08,94046.4,2909,4790.5,2906.36,1551.0,829956.92,402871.00,427085.92,38.7,...,39712,3420686486,2482162502,1.666667e+09,-6.010253e-01,2.5,1013904.9,73.2,34.45,10710868.68
2000-12-31,17811854.38,103213.7,3015,4679.0,3011.85,1620.0,1343692.67,664461.50,679231.17,38.6,...,40401,974763521,1734991942,4.285714e+09,4.748355e+09,2.2,1238144.2,73.6,37.90,11628028.20


# Visualización de las series temporales originales dentro de X

In [ ]:
# Visualización
for columnas in X.columns:
  plt.figure(figsize=(8,4))
  plt.plot(X[columnas])
  plt.title(columnas)
  plt.show()

Output hidden; open in https://colab.research.google.com to view.

# Crear archivo json con el diccionario para definir que tipo de interpolación requiere la serie de tiempo

In [ ]:
# === 3. Heuristica para definir las series que deben llevar log-transform ===
def candidatos_para_log(df, umbral_ratio=5, umbral_media=1000):
    """
    Detecta automáticamente columnas candidatas para log-transform.

    Requisitos:
    - No deben tener ceros o negativos.
    - Deben tener una razón entre máximo y mínimo significativa.
    - Deben tener un valor medio razonablemente alto.

    Retorna:
    --------
    Lista de nombres de columnas candidatas.
    """
    candidatos = []
    for col in df.columns:
        serie = pd.to_numeric(df[col], errors='coerce').dropna()
        if (serie <= 0).any():
            continue  # ❌ No apta para log
        if len(serie) < 3:
            continue  # No suficiente información
        rango = serie.max() / serie.min()
        if rango > umbral_ratio and serie.mean() > umbral_media:
            candidatos.append(col)
    return candidatos

In [ ]:
# Uso de candidatos_para_log
series_log = candidatos_para_log(X)
print("🔍 Series candidatas para log-transform:")
for s in series_log:
    print("-", s)

🔍 Series candidatas para log-transform:
- Producción bruta del sector energetico, (Millones de pesos a precios corrientes).
- Exportacion de petroleo crudo, (Miles de dolares )
- Importación de mercancias, Petroleras, (Millones de dólares)
- Generación bruta de energia eléctrica, Termoeléctrica- Vapor (Gigawatts/Hora)
- Generación bruta de energia eléctrica, Termoeléctrica- Turbogas (Gigawatts/Hora)
- Generación bruta de energia eléctrica, Termoeléctrica-Combustion interna (Gigawatts/Hora)
- Generación bruta de energia eléctrica, Termoeléctrica-ciclo combinado (Gigawatts/Hora)
- Gasto Publico


In [ ]:
# Generar la configuración de interpolación para cada serie
def generar_config_fija_log4(df, series_log):
    """
    Genera un diccionario de configuración para interpolación:
    - Aplica interpolación lineal escalonada a todas las series.
    - Solo a las variables incluidas en `series_log` se les aplica log-transform.

    Parámetros:
    -----------
    df : pd.DataFrame
        DataFrame con todas las variables.
    series_log : list
        Lista con nombres exactos de las variables que deben recibir log-transform.

    Retorna:
    --------
    dict con claves por variable y valores con las claves:
        {metodo, escalonado, log_transform}
    """
    config_auto = {}
    for col in df.columns:
        config_auto[col] = {
            "metodo": "linear",
            "escalonado": True,
            "log_transform": col in series_log
        }
    return config_auto

In [ ]:
# Uso de generar_config_fija_log4
config_dict = generar_config_fija_log4(X, series_log)

In [ ]:
# ===  Guardar configuración generada a archivo JSON ===
# Nombre del archivo json
nombre = "config_interpolacion.json"

def guardar_config_json(config_dict, ruta_json):
    """
    Guarda el diccionario de configuración como archivo JSON.

    Parámetros:
    -----------
    config_dict : dict
        Diccionario de configuración generado con generar_config_fija_log4().
    ruta_json : str
        Ruta completa donde guardar el archivo JSON.
    """
    with open(ruta_json, "w") as f:
        json.dump(config_dict, f, indent=2)
    print(f"✅ Archivo de configuración guardado en: {ruta_json}")

In [ ]:
guardar_config_json(config_dict, ruta + nombre)

✅ Archivo de configuración guardado en: /content/drive/MyDrive/Colab Notebooks/Energy thesis/Pytorch/Codigos finales tesis/Bases de datos originales/config_interpolacion.json


# Interpolación de las series originales (anuales) a diarias

In [ ]:
# === 4. Función principal de interpolación segura y escalonada para cada serie ===
def interpolar_escalonado(serie_original, frecuencia_final='D', metodo='linear', escalonado=True, log_transform=False, revert_log=True):
    """
    Interpola una serie temporal desde frecuencia anual hacia una frecuencia más alta,
    con opción de hacerlo en pasos (escalonado) o directamente.

    Parámetros:
    -----------
    serie_original : pd.Series
        Serie temporal con índice datetime y frecuencia anual.
    frecuencia_final : str
        Frecuencia de destino: 'Q' (trimestral), 'M' (mensual), 'D' (diaria).
    metodo : str
        Método de interpolación: 'linear', 'spline', 'polynomial', 'ffill'.
    escalonado : bool
        Si True, realiza interpolaciones intermedias (ej. anual→trimestral→mensual→diaria).
    log_transform : bool
        Si True, aplica log-transform antes de interpolar.
    revert_log : bool
        Si True, aplica np.exp() al final. Si False, conserva la escala logarítmica.

    Retorna:
    --------
    pd.Series con índice en frecuencia_final e interpolada.
    """
    if not isinstance(serie_original, pd.Series):
        raise ValueError("La entrada debe ser una pd.Series")

    serie = serie_original.copy()
    serie.index = pd.to_datetime(serie.index)
    serie = serie.sort_index()

    pasos = ['A', 'Q', 'M', 'D']
    if frecuencia_final not in pasos:
        raise ValueError(f"Frecuencia destino '{frecuencia_final}' no válida. Usa una de: {pasos}")
    target_idx = pasos.index(frecuencia_final)

    freq_inferida = pd.infer_freq(serie.index)
    if not freq_inferida or freq_inferida[0] != 'A':
        print("⚠️ Advertencia: frecuencia inicial no parece ser anual.")

    if metodo == 'ffill':
        nueva_frecuencia = pd.date_range(start=serie.index.min(), end=serie.index.max(), freq=frecuencia_final)
        serie = serie.reindex(nueva_frecuencia).ffill()
        serie.index.name = 'fecha'
        return serie

    if log_transform:
        if (serie <= 0).any():
            raise ValueError("La serie contiene ceros o negativos: no se puede aplicar log-transform.")
        serie = np.log(serie)

    for paso in pasos[1:target_idx+1] if escalonado else [frecuencia_final]:
        idx_nuevo = pd.date_range(start=serie.index.min(), end=serie.index.max(), freq=paso)
        serie = serie.reindex(idx_nuevo)

        if metodo in ['spline', 'polynomial'] and serie.notna().sum() < 4:
            raise ValueError(f"Interpolación con '{metodo}' requiere al menos 4 puntos no nulos")

        if metodo == 'polynomial':
            serie = serie.interpolate(method=metodo, order=2)
        elif metodo == 'spline':
            serie = serie.interpolate(method=metodo, order=3)
        else:
            serie = serie.interpolate(method=metodo)

    if log_transform and revert_log:
        serie = np.exp(serie)

    serie.index.name = 'fecha'
    return serie



In [ ]:
# === 4.1 Aplicar interpolación a todas las columnas según archivo de configuración ===
def aplicar_interpolacion_config(df_original, config_path, frecuencia_final='D', override_escalonado=None):
    """
    Aplica interpolación a cada columna según configuración en JSON.

    Parámetros:
    -----------
    df_original : pd.DataFrame
        DataFrame con las series originales (frecuencia anual).
    config_path : str
        Ruta al archivo JSON.
    frecuencia_final : str
        Frecuencia destino: 'D', 'M', 'Q'.
    override_escalonado : bool or None
        Si se indica, reemplaza 'escalonado' de cada serie.

    Retorna:
    --------
    pd.DataFrame interpolado.
    """
    with open(config_path, "r") as f:
        config = json.load(f)

    df_interpolado = pd.DataFrame()

    for col in df_original.columns:
        if col not in config:
            print(f"⚠️ Serie '{col}' no está en el JSON. Saltando.")
            continue

        serie = df_original[col].copy()
        parametros = config[col]

        try:
            serie_interp = interpolar_escalonado(
                serie,
                frecuencia_final=frecuencia_final,
                metodo=parametros.get("metodo", "linear"),
                escalonado=override_escalonado if override_escalonado is not None else parametros.get("escalonado", True),
                log_transform=parametros.get("log_transform", False)
            )
            df_interpolado[col] = serie_interp

        except Exception as e:
            print(f"❌ Error al interpolar '{col}': {e}")

    return df_interpolado



In [ ]:
# Aplicar interpolación a todas las series
ruta_json = ruta + "/config_interpolacion.json"
series_multicanal_diario = aplicar_interpolacion_config(
    X,
    config_path=ruta_json,
    frecuencia_final='D',  # o 'M', 'Q'
    override_escalonado=True  # ✅ Forzar escalonado en todas si se desea
)

⚠️ Advertencia: frecuencia inicial no parece ser anual.
⚠️ Advertencia: frecuencia inicial no parece ser anual.
⚠️ Advertencia: frecuencia inicial no parece ser anual.
⚠️ Advertencia: frecuencia inicial no parece ser anual.
⚠️ Advertencia: frecuencia inicial no parece ser anual.
⚠️ Advertencia: frecuencia inicial no parece ser anual.
⚠️ Advertencia: frecuencia inicial no parece ser anual.
⚠️ Advertencia: frecuencia inicial no parece ser anual.
⚠️ Advertencia: frecuencia inicial no parece ser anual.
⚠️ Advertencia: frecuencia inicial no parece ser anual.
⚠️ Advertencia: frecuencia inicial no parece ser anual.
⚠️ Advertencia: frecuencia inicial no parece ser anual.
⚠️ Advertencia: frecuencia inicial no parece ser anual.
⚠️ Advertencia: frecuencia inicial no parece ser anual.
⚠️ Advertencia: frecuencia inicial no parece ser anual.
⚠️ Advertencia: frecuencia inicial no parece ser anual.
⚠️ Advertencia: frecuencia inicial no parece ser anual.
⚠️ Advertencia: frecuencia inicial no parece ser

In [ ]:
series_multicanal_diario.head()

,PIB a precios corrientes (Millones de pesos),"Producción bruta del sector energetico, (Millones de pesos a precios corrientes).",Producción de petróleo crudo (Miles de barriles por día),Producción de gas natural (Millones de pies cúbicos diarios),"Producción de hidrocarburos liquidos, (Miles de barriles diarios)",Exportaciones de petróleo crudo (Miles de barriles diarios),"Exportacion de petroleo crudo, (Miles de dolares )","Importación de mercancias, Petroleras, (Millones de dólares)",Balanza comercial petrolera (Millones de dolares),"índice de volumne fisico, electricidad, gas y agua (indices)",...,"Generación bruta de energia eléctrica, Renovable (Gigawatts/Hora)",Tasa de crecimento de sector energetico,"Tasa de crecimento de sector energetico , energias renovables","Tasa de crecimento de sector energetico, Eolica","Tasa de crecimento de sector energetico, Geotermica",Tasa de Desempleo/%,Gasto Publico,Esperanza de Vida,Salario Minimo/ $,Consumo Privado
fecha,,,,,,,,,,,,,,,,,,,,,
1996-12-31,1.449913e+07,47497.600000,2861.000000,4194.900000,2858.310000,1552.000000,984742.580000,256019.080000,728723.500000,37.200000,...,38580.000000,4.084129e+09,7.970447e+09,-1.666667e+09,1.058388e+09,5.500000,581532.900000,72.000000,20.150000,8.916935e+06
1997-01-01,1.450193e+07,47525.622381,2861.440860,4195.631720,2858.749839,1552.481183,984660.343512,256267.682836,728343.261882,37.199731,...,38565.438172,4.079746e+09,7.945246e+09,-1.662186e+09,1.043202e+09,5.495161,581925.091799,72.001344,20.166935,8.918805e+06
1997-01-02,1.450474e+07,47553.661294,2861.881720,4196.363441,2859.189677,1552.962366,984578.113892,256516.527074,727963.023763,37.199462,...,38550.876344,4.075362e+09,7.920046e+09,-1.657706e+09,1.028016e+09,5.490323,582317.548096,72.002688,20.183871,8.920674e+06
1997-01-03,1.450755e+07,47581.716749,2862.322581,4197.095161,2859.629516,1553.443548,984495.891139,256765.612948,727582.785645,37.199194,...,38536.314516,4.070978e+09,7.894846e+09,-1.653226e+09,1.012831e+09,5.485484,582710.269070,72.004032,20.200806,8.922544e+06
1997-01-04,1.451035e+07,47609.788757,2862.763441,4197.826882,2860.069355,1553.924731,984413.675253,257014.940692,727202.547527,37.198925,...,38521.752688,4.066594e+09,7.869645e+09,-1.648746e+09,9.976451e+08,5.480645,583103.254898,72.005376,20.217742,8.924413e+06


In [ ]:
series_multicanal_diario.shape

(9862, 67)

# Visualización final entre series originales vs series interpoladas

In [ ]:
# === 5 Función para visualizar interpolación comparada con la serie original ===
def comparar_interpolacion_vs_original(serie_original, serie_interpolada, nombre="", color_linea='tab:blue'):
    """
    Visualiza la interpolación respecto a los puntos originales para verificar que
    la interpolación mantenga la tendencia general sin artefactos.
    """
    plt.figure(figsize=(12, 5))
    plt.plot(serie_interpolada, label='Interpolada', color=color_linea, linewidth=1.5)
    plt.scatter(serie_original.index, serie_original.values, label='Original (anual)', color='black', zorder=5, marker='o')
    plt.title(f"Comparación: {columna}", fontsize=14)
    plt.xlabel("Fecha")
    plt.ylabel("Valor")
    plt.grid(True, linestyle='--', alpha=0.5)
    plt.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
# Uso de comparar_interpolacion_vs_original(
for columna in series_multicanal_diario.columns:
  comparar_interpolacion_vs_original(X[columna], series_multicanal_diario[columna])

Output hidden; open in https://colab.research.google.com to view.

# Guardamos las series interpoladas en un archivo excel

In [ ]:
# Asignamos nombre al índice temporal
series_multicanal_diario.index.name = 'fecha'

# Guardamos el archivo
ruta_guardado ='/content/drive/MyDrive/Colab Notebooks/Energy thesis/Pytorch/Codigos finales tesis/Bases_datos_finales_modelostesis/'
series_multicanal_diario.to_excel(ruta_guardado + 'matriz_energetica_economica_diario.xlsx', index=True)


# Anexos

In [ ]:
# prompt: revisar que no hay Nan en series_multicanal_diario

# === 6. Verificar NaNs en series_multicanal_diario ===
print("--- Verificación de NaNs en series_multicanal_diario ---")
nan_counts = series_multicanal_diario.isnull().sum()
print("Número de NaNs por columna:")
print(nan_counts[nan_counts > 0]) # Mostrar solo columnas con NaNs

if nan_counts.sum() == 0:
    print("✅ ¡No se encontraron NaNs en series_multicanal_diario!")
else:
    print(f"❌ Se encontraron {nan_counts.sum()} NaNs en total.")
    print("Las columnas con NaNs son:")
    print(nan_counts[nan_counts > 0].index.tolist())

# Opcional: Visualizar NaNs (puede ser lento para dataframes grandes)
# import missingno as msno
# print("\nVisualizando NaNs:")
# msno.matrix(series_multicanal_diario)
# plt.title("Distribución de NaNs en series_multicanal_diario")
# plt.show()

# msno.bar(series_multicanal_diario)
# plt.title("Conteo de NaNs por columna en series_multicanal_diario")
# plt.show()

--- Verificación de NaNs en series_multicanal_diario ---
Número de NaNs por columna:
Series([], dtype: int64)
✅ ¡No se encontraron NaNs en series_multicanal_diario!
